## This model constuct a networkx graph from data using the following nodes

### Nodes:
- City: aggregate all stations within a city

### Edges:
- BikeTrip: bike trip between cities

In [1]:
import pandas as pd
import networkx as nx
from graphgen import fix_columns, create_graph

In [2]:
trips_filename = '../data//201508_trip_data.csv'
stations_filename = '../data/201508_station_data.csv'

trips_df = pd.read_csv(trips_filename)
stations_df = pd.read_csv(stations_filename)

# if columns have spaces in their names we need to replace them with underscore
fix_columns(trips_df)
fix_columns(stations_df)

In [7]:
display(stations_df.head())

,station_id,name,lat,long,dockcount,landmark,installation
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


In [3]:
display(stations_df['landmark'].unique())

array(['San Jose', 'Redwood City', 'Mountain View', 'Palo Alto',
       'San Francisco'], dtype=object)

In [4]:
trips_df.loc[0,'Start_Terminal']

50

In [5]:
display(stations_df[stations_df.station_id == trips_df.loc[0, 'Start_Terminal']]['landmark'])

39    San Francisco
Name: landmark, dtype: object

In [37]:
city_df = stations_df[['station_id', 'landmark']]
city_df.set_index('station_id', inplace=True)

In [42]:
# city_df.set_index('station_id', inplace=True)
landmark_dict = city_df['landmark'].to_dict()

In [ ]:
station_mapper = {
    'nodes': [
        {
            'type'      : 'City',
            'key'       : 'city_id',
            'attributes': ['landmark']
        },
    ]
}

edges_mapper = {
    'edges': [
        {
            'type'      : 'BikeTrip',
            'from'      : {
                'key'     :'city_id',
                'node_key': 'city_id'
            },
            'to'        : {
                'key'     : 'city_id',
                'node_key': 'city_id'
            },
            'attributes': ['Trip_ID', 'Start_Date', 'End_Date', 'Duration']
        }
    ]
}